In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd


print(tf.__version__)

2.15.0


In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
#loading dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


### Model VGG-7


In [ ]:
#normalzing data using maximum grey-scale value
input_shape = (28, 28, 1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

In [ ]:
#one hot encoding
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
batch_size = 64
num_classes = 10
num_epochs = 30

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=input_shape, name = 'conv1_1'),
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', name = 'conv1_2'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', name = 'conv2_1'),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', name = 'conv2_2'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', name = 'FC_1'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', name = 'FC_2'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax', name = 'FC_softmax')
])

model.compile(optimizer=tf.keras.optimizers.SGD(name = 'SGD'), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
optimizer = tf.keras.optimizers.SGD(name = 'SGD')
cce = tf.keras.losses.CategoricalCrossentropy()

Function to compute average norm gradients per layer

In [ ]:
def avg_grd_norms(variables, gradients):
  # take norm of each gradient, average them, and normalize with sqrt(no_gradients)
  norms = []
  names = []
  index = 0
  for var, grad in zip(model.trainable_variables, gradients):
      layer_size = grad.shape[-1]
      normalizer = np.sqrt(grad.shape[-1])
      # index 0,2,4,6 are conv layers
      if index in [0,2,4,6]:
          l2 = []
          for i in range(layer_size):
              flattened_weights = tf.reshape(grad[:,:,:,i], [-1])
              normalized_norm = tf.norm(flattened_weights, ord = 'euclidean')/normalizer
              l2.append(normalized_norm)

          norms.append(sum(l2))
          names.append(var.name)
          index += 1
      #index 8,10,12 are fully connected layer
      elif index in [8 ,10, 12]:
          l2 = []
          for i in range(layer_size):
              flattened_weights = tf.reshape(grad[:,i], [-1])
              normalized_norm = tf.norm(flattened_weights, ord = 'euclidean')/normalizer
              l2.append(normalized_norm)
          norms.append(normalized_norm)
          names.append(var.name)
          index += 1
      # remaining indices are bias weights
      else:
          index += 1

  return (names, norms)

In [ ]:
# names, norms = avg_grd_norms(model.trainable_variables, gradients)


In [ ]:
# tf.compat.v1.enable_eager_execution()

In [ ]:
# noisy_norms = []
# for norm, name in zip(norms, names):
#   norm = norm.numpy()
#   noisy_norms.append((name, norm))

# print(noisy_norms)

In [ ]:
# import csv

# file_path = '/content/drive/MyDrive/noisy_norms.csv'

# with open(file_path, 'w', newline='') as csvfile:

#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow(['name', 'n_norm'])

#     csv_writer.writerows(noisy_norms)

# print(f"CSV file saved at: {file_path}")

### Training, logging gradients per epoch and layer

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy

accuracy_metric = CategoricalAccuracy()

epoch_gradients = []
# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for batch_start in range(0, len(x_train), batch_size):
        # Extract a batch of data
        X_batch = x_train[batch_start:batch_start + batch_size]
        y_batch = y_train[batch_start:batch_start + batch_size]

        # Forward pass and compute gradients
        with tf.GradientTape() as tape:
            predictions = model(X_batch)
            loss = cce(y_batch, predictions)
            loss_value = tf.reduce_mean(loss)

        # Backward pass and update weights
        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # Get accuracy of the epoch
        accuracy_metric.update_state(y_batch, predictions)

    accuracy = accuracy_metric.result()
    accuracy_metric.reset_states() #reset for next epoch

    epoch_gradients.append(gradients)

    print(f"  Loss: {loss_value.numpy()}, Acc: {accuracy.numpy()}")


Epoch 1/30


  Loss: 0.04759138077497482, Acc: 0.7436166405677795
Epoch 2/30
  Loss: 0.017704876139760017, Acc: 0.9591666460037231
Epoch 3/30
  Loss: 0.015902332961559296, Acc: 0.9739500284194946
Epoch 4/30
  Loss: 0.009681770578026772, Acc: 0.9798833131790161
Epoch 5/30
  Loss: 0.007226796355098486, Acc: 0.9830499887466431
Epoch 6/30
  Loss: 0.004861659370362759, Acc: 0.98580002784729
Epoch 7/30
  Loss: 0.004084073472768068, Acc: 0.9878666400909424
Epoch 8/30
  Loss: 0.002802059054374695, Acc: 0.9896166920661926
Epoch 9/30
  Loss: 0.0027494197711348534, Acc: 0.9909999966621399
Epoch 10/30
  Loss: 0.0015986025100573897, Acc: 0.9920499920845032
Epoch 11/30
  Loss: 0.0012431354261934757, Acc: 0.9932000041007996
Epoch 12/30
  Loss: 0.0011399041395634413, Acc: 0.9940000176429749
Epoch 13/30
  Loss: 0.0010100568179041147, Acc: 0.9949833154678345
Epoch 14/30
  Loss: 0.0008364590466953814, Acc: 0.9957000017166138
Epoch 15/30
  Loss: 0.0012466326588764787, Acc: 0.9964166879653931
Epoch 16/30
  Loss: 0.0012

In [ ]:
gradients_df = pd.DataFrame(columns = ['name'])
for index in range(0,30):
  column = f'epoch_{index}'
  gradients = epoch_gradients[index]
  names, norms = avg_grd_norms(model.trainable_variables, gradients)

  gradients_df[column] = norms

gradients_df['name'] = names



In [ ]:
gradients_df['average'] = gradients_df.iloc[:, 1:].mean(axis=1)

In [ ]:
gradients_df

,name,epoch_0,epoch_1,epoch_2,epoch_3,epoch_4,epoch_5,epoch_6,epoch_7,epoch_8,...,epoch_21,epoch_22,epoch_23,epoch_24,epoch_25,epoch_26,epoch_27,epoch_28,epoch_29,average
0,conv1_1/kernel:0,"tf.Tensor(0.09805668, shape=(), dtype=float32)","tf.Tensor(0.03826977, shape=(), dtype=float32)","tf.Tensor(0.034466155, shape=(), dtype=float32)","tf.Tensor(0.02331265, shape=(), dtype=float32)","tf.Tensor(0.021603808, shape=(), dtype=float32)","tf.Tensor(0.015770247, shape=(), dtype=float32)","tf.Tensor(0.012228814, shape=(), dtype=float32)","tf.Tensor(0.008627512, shape=(), dtype=float32)","tf.Tensor(0.008021143, shape=(), dtype=float32)",...,"tf.Tensor(0.006732892, shape=(), dtype=float32)","tf.Tensor(0.017190237, shape=(), dtype=float32)","tf.Tensor(0.012881954, shape=(), dtype=float32)","tf.Tensor(0.0080980295, shape=(), dtype=float32)","tf.Tensor(0.005120547, shape=(), dtype=float32)","tf.Tensor(0.006636835, shape=(), dtype=float32)","tf.Tensor(0.0035180834, shape=(), dtype=float32)","tf.Tensor(0.00058969413, shape=(), dtype=float32)","tf.Tensor(0.0003597979, shape=(), dtype=float32)",0.015002
1,conv1_2/kernel:0,"tf.Tensor(0.16638893, shape=(), dtype=float32)","tf.Tensor(0.080064684, shape=(), dtype=float32)","tf.Tensor(0.08079314, shape=(), dtype=float32)","tf.Tensor(0.04646801, shape=(), dtype=float32)","tf.Tensor(0.041397918, shape=(), dtype=float32)","tf.Tensor(0.029771427, shape=(), dtype=float32)","tf.Tensor(0.023187526, shape=(), dtype=float32)","tf.Tensor(0.016540185, shape=(), dtype=float32)","tf.Tensor(0.01561765, shape=(), dtype=float32)",...,"tf.Tensor(0.017256955, shape=(), dtype=float32)","tf.Tensor(0.041543216, shape=(), dtype=float32)","tf.Tensor(0.034471557, shape=(), dtype=float32)","tf.Tensor(0.019429864, shape=(), dtype=float32)","tf.Tensor(0.011151361, shape=(), dtype=float32)","tf.Tensor(0.01490381, shape=(), dtype=float32)","tf.Tensor(0.007894915, shape=(), dtype=float32)","tf.Tensor(0.0014797713, shape=(), dtype=float32)","tf.Tensor(0.00090516434, shape=(), dtype=float32)",0.031974
2,conv2_1/kernel:0,"tf.Tensor(0.22161627, shape=(), dtype=float32)","tf.Tensor(0.1349845, shape=(), dtype=float32)","tf.Tensor(0.14522828, shape=(), dtype=float32)","tf.Tensor(0.09602877, shape=(), dtype=float32)","tf.Tensor(0.0903318, shape=(), dtype=float32)","tf.Tensor(0.06541248, shape=(), dtype=float32)","tf.Tensor(0.05236018, shape=(), dtype=float32)","tf.Tensor(0.036736067, shape=(), dtype=float32)","tf.Tensor(0.036392257, shape=(), dtype=float32)",...,"tf.Tensor(0.035193715, shape=(), dtype=float32)","tf.Tensor(0.096433714, shape=(), dtype=float32)","tf.Tensor(0.07612019, shape=(), dtype=float32)","tf.Tensor(0.04383294, shape=(), dtype=float32)","tf.Tensor(0.028332721, shape=(), dtype=float32)","tf.Tensor(0.035814494, shape=(), dtype=float32)","tf.Tensor(0.020556448, shape=(), dtype=float32)","tf.Tensor(0.0036736364, shape=(), dtype=float32)","tf.Tensor(0.0021932046, shape=(), dtype=float32)",0.063183
3,conv2_2/kernel:0,"tf.Tensor(0.42489728, shape=(), dtype=float32)","tf.Tensor(0.2516552, shape=(), dtype=float32)","tf.Tensor(0.26652518, shape=(), dtype=float32)","tf.Tensor(0.16023517, shape=(), dtype=float32)","tf.Tensor(0.13869114, shape=(), dtype=float32)","tf.Tensor(0.10453052, shape=(), dtype=float32)","tf.Tensor(0.08828906, shape=(), dtype=float32)","tf.Tensor(0.06260025, shape=(), dtype=float32)","tf.Tensor(0.06820219, shape=(), dtype=float32)",...,"tf.Tensor(0.075540915, shape=(), dtype=float32)","tf.Tensor(0.2122778, shape=(), dtype=float32)","tf.Tensor(0.171138, shape=(), dtype=float32)","tf.Tensor(0.09521768, shape=(), dtype=float32)","tf.Tensor(0.059963275, shape=(), dtype=float32)","tf.Tensor(0.08129676, shape=(), dtype=float32)","tf.Tensor(0.045109496, shape=(), dtype=float32)","tf.Tensor(0.007908137, shape=(), dtype=float32)","tf.Tensor(0.004942865, shape=(), dtype=float32)",0.128413
4,FC_1/kernel:0,"tf.Tensor(0.0030817736, shape=(), dtype=float32)","tf.Tensor(0.004244768, shape=(), dtype=float32)","tf.Tensor

In [ ]:
csv_file_path = "/content/drive/MyDrive/clean_gradients.csv"

gradients_df.to_csv(csv_file_path, index = False)

print(f'Dataframe saved as CSV on google drive at {csv_file_path}')


Dataframe saved as CSV on google drive at /content/drive/MyDrive/clean_gradients.csv
